In [ ]:
import pandas as pd
import time
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

results_lock = Lock()


In [ ]:
vk_df = pd.read_excel('vk_videos_dataset_extended.xlsx')


Загружено 15444 видео из VK датасета
Столбцы: ['video_id', 'owner_id', 'title', 'description', 'duration', 'views', 'likes', 'comments', 'reposts', 'date', 'date_published', 'image', 'first_frame', 'width', 'height', 'player', 'access_key', 'platform', 'search_query', 'category', 'engagement_rate', 'duration_minutes', 'comments_per_1k_views']

Первые несколько строк:


,video_id,owner_id,title,description,duration,views,likes,comments,reposts,date,...,width,height,player,access_key,platform,search_query,category,engagement_rate,duration_minutes,comments_per_1k_views
0,456252662,-227034863,Лучшие Треки! Мощные Красивая и Потрясающяя М...,ЦИФЕi вконтакте))) http://vk.com/public2631033...,948,1757891,1504,68,123,1724422482,...,1280,720,https://vk.com/video_ext.php?oid=-227034863&id...,NaN,VK,музыка,Музыка,0.09,15.80,0.04
1,456252737,-227034863,Лучшее! Подборка Безумно Мощной и Красивой муз...,★-★-★-★-★-★-★-★-★-★-★-★-★-★-★-★-★-★-★-★\nЦИФЕi...,722,671282,722,21,34,1724422485,...,1280,720,https://vk.com/video_ext.php?oid=-227034863&id...,NaN,VK,музыка,Музыка,0.11,12.03,0.03
2,456252731,-227034863,1 ЧАС ♫ Мощная Безумно Красивая и Потрясающая ...,АЛЬБОМ Rhythm Magic https://music.yoola.com/G7...,3876,898706,1427,93,128,1724422484,...,1280,720,https://vk.com/video_ext.php?oid=-227034863&id...,NaN,VK,музыка,Музыка,0.16,64.60,0.10
3,456253112,-227034863,КЛАССНАЯ УБОЙНАЯ МУЗЫКА В МАШИНУ! ШИКАРНЫЕ ТРЕ...,ТРЕКЛИСТ:\n00:00 ЦИФЕi (Ex CEPHEI MC) - На Кра...,1399,776468,849,25,66,1724422494,...,1920,1080,https://vk.com/video_ext.php?oid=-227034863&id...,NaN,VK,музыка,Музыка,0.11,23.32,0.03
4,456253174,-227034863,ПОИСТИНЕ ПОТРЯСАЮЩАЯ КРАСИВАЯ МУЗЫКА! КРАСОТА ...,"ТРЕКЛИСТ , СКАЧАТЬ:\n00:00 ЦИФЕi - В прорыв по...",3890,91202,39,0,4,1724422496,...,1920,1080,https://vk.com/video_ext.php?oid=-227034863&id...,NaN,VK,музыка,Музыка,0.04,64.83,0.00


In [ ]:
def setup_driver(headless=True, thread_id=None):
    chrome_options = Options()
    if headless:
        chrome_options.add_argument('--headless=new')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-plugins')
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_settings.popups": 0,
        "profile.managed_default_content_settings.media_stream": 2,
    }
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation", "enable-logging"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument('--disable-background-timer-throttling')
    chrome_options.add_argument('--disable-backgrounding-occluded-windows')
    chrome_options.add_argument('--disable-renderer-backgrounding')
    chrome_options.add_argument('--disable-features=TranslateUI')
    chrome_options.add_argument('--disable-ipc-flooding-protection')
    chrome_options.add_argument('--disable-hang-monitor')
    chrome_options.add_argument('--disable-prompt-on-repost')
    chrome_options.add_argument('--disable-domain-reliability')
    chrome_options.add_argument('--disable-component-update')
    chrome_options.add_argument('--disable-sync')
    chrome_options.add_argument('--metrics-recording-only')
    chrome_options.add_argument('--no-first-run')
    chrome_options.add_argument('--safebrowsing-disable-auto-update')
    chrome_options.add_argument('--disable-default-apps')
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.set_page_load_timeout(30)
    driver.implicitly_wait(3)
    return driver


In [ ]:
def parse_number(text):
    if not text or pd.isna(text):
        return 0
    text = str(text).strip().upper()
    text = re.sub(r'[^0-9.KMB]', '', text)
    if not text:
        return 0
    try:
        if 'B' in text:
            number = float(re.sub(r'[^0-9.]', '', text)) * 1_000_000_000
        elif 'M' in text:
            number = float(re.sub(r'[^0-9.]', '', text)) * 1_000_000
        elif 'K' in text:
            number = float(re.sub(r'[^0-9.]', '', text)) * 1_000
        else:
            number = float(re.sub(r'[^0-9.]', '', text))
        return int(number)
    except:
        return 0


In [ ]:
def parse_duration(duration_text):
    if not duration_text or pd.isna(duration_text):
        return 0
    duration_text = str(duration_text).strip()
    parts = duration_text.split(':')
    try:
        if len(parts) == 3:
            hours, minutes, seconds = map(int, parts)
            return hours * 3600 + minutes * 60 + seconds
        elif len(parts) == 2:
            minutes, seconds = map(int, parts)
            return minutes * 60 + seconds
        elif len(parts) == 1:
            return int(parts[0])
        else:
            return 0
    except:
        return 0

def calculate_title_similarity(title1, title2):
    if not title1 or not title2:
        return 0.0
    title1 = str(title1).lower().strip()
    title2 = str(title2).lower().strip()
    if title1 == title2:
        return 1.0
    title1_clean = re.sub(r'[^\w\s]', '', title1)
    title2_clean = re.sub(r'[^\w\s]', '', title2)
    if title1_clean == title2_clean:
        return 0.95
    words1 = set(title1_clean.split())
    words2 = set(title2_clean.split())
    if not words1 or not words2:
        return 0.0
    intersection = len(words1 & words2)
    union = len(words1 | words2)
    jaccard = intersection / union if union > 0 else 0.0
    words_ratio = intersection / len(words1) if words1 else 0.0
    similarity = (jaccard * 0.6 + words_ratio * 0.4)
    if len(title1) > 10 and len(title2) > 10:
        max_common = 0
        for i in range(len(title1)):
            for j in range(len(title2)):
                k = 0
                while (i + k < len(title1) and j + k < len(title2) and 
                       title1[i + k] == title2[j + k]):
                    k += 1
                max_common = max(max_common, k)
        if max_common > 10:
            substring_bonus = min(max_common / max(len(title1), len(title2)), 0.2)
            similarity += substring_bonus
    return min(similarity, 1.0)

def extract_video_info_from_search_result(element):
    try:
        info = {}
        link = element.find_element(By.CSS_SELECTOR, 'a[href*="/watch?v="]')
        if link:
            video_url = link.get_attribute('href')
            if video_url:
                video_id_match = re.search(r'[?&]v=([^&]+)', video_url)
                if video_id_match:
                    info['url'] = f"https://www.youtube.com/watch?v={video_id_match.group(1)}"
                else:
                    info['url'] = video_url
        else:
            return None
        try:
            title_elem = element.find_element(By.CSS_SELECTOR, 'a#video-title, a#video-title-link')
            info['title'] = title_elem.get_attribute('title') or title_elem.text
        except:
            try:
                title_elem = element.find_element(By.CSS_SELECTOR, 'a[href*="/watch"]')
                info['title'] = title_elem.get_attribute('title') or title_elem.text
            except:
                info['title'] = None
        try:
            duration_elem = element.find_element(By.CSS_SELECTOR, 'span.style-scope.ytd-thumbnail-overlay-time-status-renderer, .ytd-thumbnail-overlay-time-status-renderer span')
            duration_text = duration_elem.text
            if duration_text:
                info['duration'] = parse_duration(duration_text)
            else:
                info['duration'] = None
        except:
            try:
                duration_elem = element.find_element(By.CSS_SELECTOR, 'span[aria-label*=":"], span[class*="duration"]')
                duration_text = duration_elem.text or duration_elem.get_attribute('aria-label')
                if duration_text:
                    info['duration'] = parse_duration(duration_text)
                else:
                    info['duration'] = None
            except:
                info['duration'] = None
        return info if info.get('url') else None
    except:
        return None


In [ ]:
def search_youtube_video(driver, title, original_duration=None, max_results=3):
    try:
        from urllib.parse import quote_plus
        max_retries = 2
        for attempt in range(max_retries):
            try:
                encoded_title = quote_plus(title)
                search_url = f"https://www.youtube.com/results?search_query={encoded_title}"
                try:
                    driver.get(search_url)
                except TimeoutException:
                    if attempt < max_retries - 1:
                        time.sleep(0.5)
                        continue
                    else:
                        raise
                time.sleep(0.5)
                try:
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.ID, "contents"))
                    )
                except TimeoutException:
                    if attempt < max_retries - 1:
                        time.sleep(0.5)
                        continue
                break
            except TimeoutException:
                if attempt < max_retries - 1:
                    time.sleep(0.5)
                    continue
                else:
                    raise
            except Exception:
                if attempt < max_retries - 1:
                    time.sleep(0.5)
                    continue
                else:
                    raise
        try:
            driver.execute_script("window.scrollTo(0, 800);")
            time.sleep(0.3)
        except:
            pass
        video_results = []
        try:
            containers = driver.find_elements(By.CSS_SELECTOR, 'ytd-video-renderer')[:max_results]
        except:
            containers = []
        if not containers:
            try:
                all_links = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/watch?v="]')
                seen_urls = set()
                for link in all_links[:max_results * 2]:
                    try:
                        video_url = link.get_attribute('href')
                        if video_url and '/watch?v=' in video_url:
                            video_id_match = re.search(r'[?&]v=([^&]+)', video_url)
                            if video_id_match:
                                video_id = video_id_match.group(1)
                                if video_id not in seen_urls:
                                    seen_urls.add(video_id)
                                    try:
                                        parent = link.find_element(By.XPATH, './ancestor::ytd-video-renderer[1]')
                                        if parent:
                                            info = extract_video_info_from_search_result(parent)
                                            if info:
                                                video_results.append(info)
                                    except:
                                        title_text = link.get_attribute('title') or link.text
                                        video_results.append({
                                            'url': f"https://www.youtube.com/watch?v={video_id}",
                                            'title': title_text,
                                            'duration': None
                                        })
                    except:
                        continue
            except:
                pass
        else:
            for container in containers[:max_results]:
                info = extract_video_info_from_search_result(container)
                if info:
                    video_results.append(info)
        if not video_results:
            return None
        best_match = None
        best_score = 0.0
        for result in video_results:
            score = 0.0
            if result.get('title'):
                title_similarity = calculate_title_similarity(title, result['title'])
                score += title_similarity * 0.7
            else:
                score += 0.3
            if original_duration and result.get('duration'):
                duration_diff = abs(original_duration - result['duration'])
                if duration_diff <= 5:
                    duration_score = 1.0
                elif duration_diff <= original_duration * 0.1:
                    duration_score = 0.8
                elif duration_diff <= original_duration * 0.2:
                    duration_score = 0.5
                else:
                    duration_score = max(0.0, 1.0 - (duration_diff / original_duration))
                score += duration_score * 0.3
            elif original_duration:
                score += 0.1
            else:
                score += 0.15
            if score > best_score:
                best_score = score
                best_match = result
        if best_score >= 0.4:
            return best_match['url']
        else:
            return None
    except:
        return None


In [ ]:
def extract_youtube_data(driver, video_url):
    if not video_url:
        return None
    max_retries = 2
    for attempt in range(max_retries):
        try:
            driver.get(video_url)
            time.sleep(0.5)
            try:
                driver.execute_script("window.scrollTo(0, 300);")
                time.sleep(0.2)
            except:
                pass
            break
        except TimeoutException:
            if attempt < max_retries - 1:
                time.sleep(0.5)
                continue
            else:
                return None
        except Exception:
            if attempt < max_retries - 1:
                time.sleep(0.5)
                continue
            else:
                raise
    data = {}
    video_id_match = re.search(r'[?&]v=([^&]+)', video_url)
    if video_id_match:
        data['video_id'] = video_id_match.group(1)
    else:
        data['video_id'] = None
    data['player'] = video_url
    try:
        extracted_data = driver.execute_script("""
                var result = {};
                var pageText = document.documentElement.innerHTML;
                var patterns = {
                    views: /"viewCount":(\\d+)/,
                    duration: /"lengthSeconds":(\\d+)/,
                    likes: /"likeCount":(\\d+)/,
                    comments: /"commentCount":(\\d+)/,
                    publishDate: /"publishDate":"([^"]+)"/,
                    description: /"shortDescription":"([^"]{0,3000})"/,
                    channelName: /"channelName":"([^"]+)"/,
                    ownerChannelName: /"ownerChannelName":"([^"]+)"/
                };
                for (var key in patterns) {
                    var match = pageText.match(patterns[key]);
                    if (match) {
                        if (key === 'description') {
                            result[key] = match[1].replace(/\\\\n/g, ' ').replace(/\\\\u0026/g, '&');
                        } else if (key === 'publishDate' || key === 'channelName' || key === 'ownerChannelName') {
                            result[key] = match[1];
                        } else {
                            result[key] = parseInt(match[1]);
                        }
                    }
                }
                if (result.ownerChannelName) {
                    result.channelName = result.ownerChannelName;
                }
                if (!result.likes) {
                    var likeMatch = pageText.match(/"likeCount":"(\\d+)"/);
                    if (likeMatch) result.likes = parseInt(likeMatch[1]);
                }
                return result;
            """)
        if extracted_data:
            data['views'] = extracted_data.get('views') or data.get('views')
            data['likes'] = extracted_data.get('likes') or data.get('likes')
            data['comments'] = extracted_data.get('comments') or data.get('comments')
            data['duration'] = extracted_data.get('duration') or data.get('duration')
            data['description'] = extracted_data.get('description') or data.get('description')
            data['date_published'] = extracted_data.get('publishDate') or data.get('date_published')
            data['channel_name'] = extracted_data.get('channelName') or data.get('channel_name')
    except:
        pass
    title_selectors = [
        'h1.ytd-watch-metadata yt-formatted-string',
        'h1 yt-formatted-string',
        'h1.ytd-video-primary-info-renderer',
        '#title h1',
        'h1[class*="title"]',
        'ytd-watch-metadata h1',
        '#container h1'
    ]
    data['title'] = None
    for selector in title_selectors:
        try:
            title_element = driver.find_element(By.CSS_SELECTOR, selector)
            if title_element and title_element.text:
                data['title'] = title_element.text.strip()
                break
        except:
            continue
    if not data['title']:
        try:
            title_js = driver.execute_script("""
                var title = document.querySelector('h1.ytd-watch-metadata yt-formatted-string, h1 yt-formatted-string');
                return title ? title.innerText : null;
            """)
            if title_js:
                data['title'] = title_js.strip()
        except:
            pass
    if not data.get('description'):
        try:
            description_js = driver.execute_script("""
                var desc = document.querySelector('#description yt-formatted-string, ytd-video-secondary-info-renderer yt-formatted-string');
                return desc ? (desc.innerText || desc.textContent) : null;
            """)
            if description_js:
                data['description'] = description_js.strip()[:1000]
        except:
            data['description'] = None
    if not data.get('views'):
        try:
            views_js = driver.execute_script("""
                var el = document.querySelector('span.view-count, yt-formatted-string[class*="view-count"]');
                return el ? (el.innerText || el.textContent) : null;
            """)
            if views_js:
                data['views'] = parse_number(views_js)
        except:
            pass
    if not data.get('likes'):
        try:
            likes_js = driver.execute_script("""
                var btn = document.querySelector('ytd-toggle-button-renderer[target-id="watch-like"] button');
                if (btn) {
                    var label = btn.getAttribute('aria-label');
                    if (label) {
                        var match = label.match(/([\\d,]+)/);
                        if (match) return match[1].replace(/,/g, '');
                    }
                }
                return null;
            """)
            if likes_js:
                try:
                    data['likes'] = int(likes_js)
                except:
                    data['likes'] = parse_number(likes_js)
        except:
            pass
    if not data.get('comments'):
        try:
            comments_js = driver.execute_script("""
                var el = document.querySelector('h2#count, #count');
                return el ? (el.innerText || el.textContent) : null;
            """)
            if comments_js:
                data['comments'] = parse_number(comments_js)
        except:
            pass
    if not data.get('duration'):
        try:
            duration_js = driver.execute_script("""
                var meta = document.querySelector('meta[itemprop="duration"]');
                if (meta) {
                    var content = meta.getAttribute('content');
                    if (content) {
                        var match = content.match(/PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?/);
                        if (match) {
                            var h = parseInt(match[1] || 0);
                            var m = parseInt(match[2] || 0);
                            var s = parseInt(match[3] || 0);
                            return h + ':' + (m < 10 ? '0' : '') + m + ':' + (s < 10 ? '0' : '') + s;
                        }
                    }
                }
                var dur = document.querySelector('span.ytp-time-duration');
                return dur ? (dur.innerText || dur.textContent) : null;
            """)
            if duration_js:
                data['duration'] = parse_duration(duration_js)
        except:
            pass
    if not data.get('date_published'):
        try:
            date_js = driver.execute_script("""
                var els = document.querySelectorAll('yt-formatted-string.style-scope.ytd-watch-info-text');
                for (var el of els) {
                    var text = (el.innerText || el.textContent || '').toLowerCase();
                    if (!text.includes('view') && text.match(/\\d{4}/)) {
                        return el.innerText || el.textContent;
                    }
                }
                return null;
            """)
            if date_js:
                data['date_published'] = date_js.strip()
        except:
            pass
    data['date'] = None
    data['width'] = None
    data['height'] = None
    data['reposts'] = None
    data['image'] = None
    data['first_frame'] = None
    data['access_key'] = None
    data['platform'] = 'YouTube'
    data['owner_id'] = None
    data['channel_name'] = None
    try:
        channel_js = driver.execute_script("""
                var pageText = document.documentElement.innerHTML;
                var match = pageText.match(/"channelName":"([^"]+)"/);
                if (match) {
                    return match[1];
                }
                match = pageText.match(/"ownerChannelName":"([^"]+)"/);
                if (match) {
                    return match[1];
                }
                return null;
            """)
        if channel_js:
            data['channel_name'] = channel_js.strip()
        if not data.get('channel_name'):
            try:
                channel_selectors = [
                    'ytd-channel-name a',
                    '#channel-name a',
                    'ytd-video-owner-renderer #channel-name a',
                    'ytd-video-owner-renderer a[href*="/channel/"]',
                    'ytd-video-owner-renderer a[href*="/@"]',
                    'ytd-channel-name yt-formatted-string',
                    '#owner-text a'
                ]
                for selector in channel_selectors:
                    try:
                        channel_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                        for channel_element in channel_elements:
                            channel_text = channel_element.text
                            if channel_text and len(channel_text.strip()) > 0:
                                data['channel_name'] = channel_text.strip()
                                break
                        if data['channel_name']:
                            break
                    except:
                        continue
            except:
                pass
        if not data.get('channel_name'):
            try:
                channel_js_dom = driver.execute_script("""
                    var channelEl = document.querySelector('ytd-channel-name a, #channel-name a, ytd-video-owner-renderer #channel-name a');
                    if (channelEl) {
                        return channelEl.innerText || channelEl.textContent;
                    }
                    return null;
                """)
                if channel_js_dom:
                    data['channel_name'] = channel_js_dom.strip()
            except:
                pass
    except:
        pass
    return data


In [ ]:
def process_video(driver, row, index, total):
    title = str(row['title']) if pd.notna(row['title']) else ''
    original_duration = None
    if pd.notna(row.get('duration')):
        try:
            original_duration = int(row['duration'])
        except:
            pass
    if DELAY_BETWEEN_VIDEOS > 0:
        time.sleep(DELAY_BETWEEN_VIDEOS)
    try:
        video_url = search_youtube_video(driver, title, original_duration=original_duration, max_results=3)
    except TimeoutException:
        video_url = None
    except Exception:
        video_url = None
    if video_url:
        youtube_data = extract_youtube_data(driver, video_url)
        if youtube_data:
            youtube_data['search_query'] = row.get('search_query', None)
            youtube_data['category'] = row.get('category', None)
            return youtube_data
    return {
        'video_id': None,
        'owner_id': None,
        'title': title,
        'description': None,
        'duration': None,
        'views': None,
        'likes': None,
        'comments': None,
        'reposts': None,
        'date': None,
        'date_published': None,
        'image': None,
        'first_frame': None,
        'width': None,
        'height': None,
        'player': None,
        'access_key': None,
        'platform': 'YouTube',
        'channel_name': None,
        'search_query': row.get('search_query', None),
        'category': row.get('category', None)
    }


In [ ]:
START_INDEX = 0
MAX_VIDEOS = None
NUM_THREADS = 5
HEADLESS_MODE = True
DELAY_BETWEEN_VIDEOS = 0.1

def process_video_worker(args):
    idx, row, total, thread_id = args
    driver = None
    try:
        driver = setup_driver(headless=HEADLESS_MODE, thread_id=thread_id)
        result = process_video(driver, row, idx, total)
        return (idx, result)
    except Exception:
        return (idx, {
            'video_id': None,
            'owner_id': None,
            'title': str(row.get('title', '')),
            'description': None,
            'duration': None,
            'views': None,
            'likes': None,
            'comments': None,
            'reposts': None,
            'date': None,
            'date_published': None,
            'image': None,
            'first_frame': None,
            'width': None,
            'height': None,
            'player': None,
            'access_key': None,
            'platform': 'YouTube',
            'channel_name': None,
            'search_query': row.get('search_query', None),
            'category': row.get('category', None)
        })
    finally:
        if driver:
            driver.quit()

try:
    total_videos = len(vk_df)
    if MAX_VIDEOS:
        total_videos = min(MAX_VIDEOS, total_videos)
    end_index = START_INDEX + total_videos
    videos_to_process = vk_df.iloc[START_INDEX:end_index]
    youtube_results = []
    processed_indices = set()
    try:
        existing_df = pd.read_excel('youtube_videos_dataset_temp.xlsx')
        if len(existing_df) > 0:
            youtube_results = existing_df.to_dict('records')
            processed_indices = set(range(len(youtube_results)))
    except:
        pass
    tasks = []
    for idx, (index, row) in enumerate(videos_to_process.iterrows()):
        if idx not in processed_indices:
            tasks.append((idx, row, len(videos_to_process), idx % NUM_THREADS))
    start_time = time.time()
    completed = 0
    new_results = {}
    with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
        future_to_task = {executor.submit(process_video_worker, task): task for task in tasks}
        for future in as_completed(future_to_task):
            try:
                idx, result = future.result()
                new_results[idx] = result
                completed += 1
                if completed % 100 == 0:
                    all_results = youtube_results.copy()
                    for i in sorted(new_results.keys()):
                        if i < len(all_results):
                            all_results[i] = new_results[i]
                        else:
                            all_results.append(new_results[i])
                    with results_lock:
                        temp_df = pd.DataFrame(all_results)
                        vk_columns = vk_df.columns.tolist()
                        temp_df = temp_df.reindex(columns=vk_columns, fill_value=None)
                        temp_df.to_excel('youtube_videos_dataset_temp.xlsx', index=False)
            except KeyboardInterrupt:
                break
            except Exception:
                pass
    final_results = []
    for idx in range(len(videos_to_process)):
        if idx in new_results:
            final_results.append(new_results[idx])
        elif idx < len(youtube_results):
            final_results.append(youtube_results[idx])
    youtube_results = final_results
    if youtube_results:
        temp_df = pd.DataFrame(youtube_results)
        vk_columns = vk_df.columns.tolist()
        temp_df = temp_df.reindex(columns=vk_columns, fill_value=None)
        temp_df.to_excel('youtube_videos_dataset_temp.xlsx', index=False)
except KeyboardInterrupt:
    if 'youtube_results' in locals() and youtube_results:
        temp_df = pd.DataFrame(youtube_results)
        vk_columns = vk_df.columns.tolist()
        temp_df = temp_df.reindex(columns=vk_columns, fill_value=None)
        temp_df.to_excel('youtube_videos_dataset_temp.xlsx', index=False)
except Exception:
    pass


🚀 Запуск оптимизированного парсера с 5 потоками
📊 Headless режим: Включен

Начинаем обработку 15444 видео...
Загружено 15444 ранее обработанных видео
📋 К обработке: 0 видео
⚡ Режим: МАКСИМАЛЬНАЯ СКОРОСТЬ (5 потоков, оптимизированные задержки)
⏱️  Примерное время: 0.0 минут (оптимизировано)

💾 Финальное сохранение: 15444 видео
⏱️  Общее время: 0.1 минут (0.00 часов)
📊 Скорость: 172456.5 видео/минуту

✅ Обработка завершена!


In [ ]:
if 'youtube_results' in locals() and len(youtube_results) > 0:
    youtube_df = pd.DataFrame(youtube_results)
    vk_columns = vk_df.columns.tolist()
    youtube_df = youtube_df.reindex(columns=vk_columns, fill_value=None)
else:
    youtube_df = pd.DataFrame()


Создан датасет с 15444 видео

Статистика:
  Найдено на YouTube: 10568
  Не найдено: 4876


In [ ]:
youtube_df.to_excel('youtube_videos_dataset.xlsx', index=False)
youtube_df.to_csv('youtube_videos_dataset.csv', index=False, encoding='utf-8-sig')


✅ Датасет сохранен в файл: youtube_videos_dataset.xlsx
✅ Датасет также сохранен в CSV: youtube_videos_dataset.csv


In [ ]:
if len(youtube_df) > 0 and youtube_df['video_id'].notna().sum() > 0:
    found_videos = youtube_df[youtube_df['video_id'].notna()]
    found_videos[['views', 'likes', 'comments', 'duration']].describe()


Статистика по найденным видео на YouTube:
Среднее количество просмотров: 12,408,803
Среднее количество лайков: 130,617
Среднее количество комментариев: nan
Средняя длительность (секунды): 1578

Топ-10 видео по просмотрам:
                                                                  title         views       likes  comments
344                    Ed Sheeran - Shape of You (Official Music Video)  6.600704e+09  34978732.0       NaN
336                         Ed Sheeran - Perfect (Official Music Video)  4.056275e+09  23141110.0       NaN
340                         Ed Sheeran - Perfect (Official Music Video)  4.056275e+09  23141111.0       NaN
347                         Ed Sheeran - Perfect (Official Music Video)  4.056275e+09  23141111.0       NaN
332               Ed Sheeran - Thinking Out Loud (Official Music Video)  3.947768e+09  15861105.0       NaN
289   The Weeknd - Starboy ft. Daft Punk (Official Video) ft. Daft Punk  2.670847e+09  12994226.0       NaN
387                   

In [ ]:
import os

df_vk = pd.read_excel('vk_videos_dataset_extended.xlsx') if os.path.exists('vk_videos_dataset_extended.xlsx') else None
df_yt = pd.read_csv('youtube_videos_dataset.csv') if os.path.exists('youtube_videos_dataset.csv') else None

In [ ]:
if df_vk is not None and df_yt is not None:
    num_rows = min(len(df_vk), len(df_yt))
    yt_mask = df_yt['video_id'].notna() if 'video_id' in df_yt.columns else pd.Series([True] * len(df_yt), index=df_yt.index)
    yt_mask = yt_mask.iloc[:num_rows]
    df_vk_selected = df_vk.iloc[:num_rows][yt_mask].copy().reset_index(drop=True)
    df_yt_selected = df_yt.iloc[:num_rows][yt_mask].copy().reset_index(drop=True)
    common_cols = set(df_vk_selected.columns) & set(df_yt_selected.columns)
    df_yt_renamed = df_yt_selected.copy()
    if common_cols:
        rename_dict = {col: f"{col}_youtube" for col in common_cols if col in df_yt_renamed.columns}
        df_yt_renamed = df_yt_renamed.rename(columns=rename_dict)
    merged_df = pd.concat([df_vk_selected, df_yt_renamed], axis=1)
else:
    merged_df = None


In [ ]:
df_merged_kat = pd.read_excel('merged_vk_youtube_dataset.xlsx')
df_merged_kat.sort_values(['views'])

In [ ]:
out = df_merged_kat.sort_values(['views']).drop_duplicates(subset=['video_id_youtube'], keep='last')
out = out.drop(out[out.duration == 0].index)
out = out.drop(out[out.duration_youtube == 0].index)
out = out.drop(out[out.duration.isna()].index)
out = out.drop(out[out.duration_youtube.isna()].index)
out = out.drop(out[((out.duration / out.duration_youtube > 1.2) | (out.duration / out.duration_youtube < 0.8))].index)
len(out)

In [ ]:
def extract_year(date_value):
    if pd.isna(date_value):
        return None
    try:
        if isinstance(date_value, str):
            for fmt in ['%Y-%m-%d', '%d.%m.%Y', '%Y/%m/%d', '%d-%m-%Y', '%Y-%m-%d %H:%M:%S']:
                try:
                    dt = datetime.strptime(date_value, fmt)
                    return dt.year
                except:
                    continue
            year_match = re.search(r'\d{4}', str(date_value))
            if year_match:
                year = int(year_match.group())
                if 1900 <= year <= 2100:
                    return year
        elif isinstance(date_value, (int, float)):
            if 1900 <= date_value <= 2100:
                return int(date_value)
            else:
                try:
                    dt = datetime.fromtimestamp(date_value)
                    return dt.year
                except:
                    pass
        else:
            dt = pd.to_datetime(date_value, errors='coerce')
            if pd.notna(dt):
                return dt.year
    except:
        pass
    return None

if 'final' in locals() and 'date_published' in final.columns:
    final['year_published_vk'] = final['date_published'].apply(extract_year)
if 'final' in locals() and 'date_published_youtube' in final.columns:
    final['year_published_youtube'] = final['date_published_youtube'].apply(extract_year)